# Train and evaluate a PC

In [5]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt

In [6]:
device = torch.device("cpu")  # The device to use, e.g., "cpu", "cuda", "cuda:1"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%reload_ext autoreload

Set the random seeds.

In [8]:
random.seed(4)
np.random.seed(4)
torch.manual_seed(4)
# if 'cuda' in device.type:
#     torch.cuda.manual_seed(42)

## Load MNIST Dataset

Load the training and test splits of MNIST, and preprocess them by flattening the tensor images.

In [ ]:
from torchvision import transforms, datasets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: (255 * x.view(-1)).long())
])
data_train = datasets.MNIST('datasets', train=True, download=True, transform=transform)
data_test = datasets.MNIST('datasets', train=False, download=True, transform=transform)
num_variables = data_train[0][0].shape[0]
height, width = 28, 28
print(f"Number of variables: {num_variables}")

In [ ]:
plt.matshow(data_train[0][0].reshape(28, 28), cmap='gray')
plt.title(f"Class: {data_train[0][1]}")
plt.show()

In [35]:
import torch.nn.functional as F

from uci_datasets import Dataset

from ignite.engine import Events, Engine
from ignite.metrics import Average, Loss
from ignite.contrib.handlers import ProgressBar

import gpytorch
from gpytorch.mlls import VariationalELBO
from gpytorch.likelihoods import GaussianLikelihood

import pandas as pd
import numpy as np


## Instantiating the region graph

In [36]:
from cirkit.region_graph.poon_domingos import PoonDomingos
from cirkit.region_graph.random_binary_tree import RandomBinaryTree
from cirkit.region_graph.fully_factorized import FullyFactorized

In [37]:
from cirkit.models.gp import CircuitGP, initial_values

Initialize a _Quad Graph_ region graph.

Others available region graphs are _Poon Domingos_ and _QuadTree_, whose imports are showed below.

## Choosing the layers

Now we have to choose both the input and inner layers of our circuit. As input layer we select the _CategoricalLayer_ with 256 categories (the number of pixel values). For the inner layer instead, we choose the _uncollapsed CP_ layer with rank 1.

In [32]:
from cirkit.layers.input.exp_family import CategoricalLayer
from cirkit.layers.sum_product import CPLayer
from cirkit.layers.input.rbf_kernel import RBFKernelLayer

efamily_cls = RBFKernelLayer
efamily_kwargs = {}
layer_cls = CPLayer
layer_kwargs = {'rank': 1}

## Building the tensorized PC

We can now build our tensorized PC by specifying the region graph and layers we chose previously. In addition, we can scale the architecture by increasing the number of input and inner units. We can also have circuits with multiple output units by choosing _num_classes > 1_. However, in this notebook we only estimate the distribution of the images and marginalize out the class variable.

To ensure weights are non-negative we reparametrize them via exponentiation. Several reparametrization functions are available.

In [89]:
from cirkit.region_graph.quad_tree import QuadTree
# region_graph = QuadTree(width, height, struct_decomp=False)
region_graph = RandomBinaryTree(num_vars=8, depth=3, num_repetitions=6)
# region_graph = FullyFactorized(num_vars=8)

In [90]:
from cirkit.reparams.leaf import ReparamExp, ReparamLogSoftmax, ReparamSoftmax
from cirkit.models.tensorized_circuit import TensorizedPC
pc = TensorizedPC.from_region_graph(
    region_graph,
    num_inner_units=1,
    num_input_units=1,
    efamily_cls=efamily_cls,
    efamily_kwargs=efamily_kwargs,
    layer_cls=layer_cls,
    layer_kwargs=layer_kwargs,
    num_classes=1,
    reparam=ReparamExp # ReparamLogSoftmax # ReparamExp ReparamSoftmax
)
pc.to(device)
print(pc)

TensorizedPC(
  (input_layer): RBFKernelLayer(
    (params): ReparamExp()
  )
  (scope_layer): ScopeLayer()
  (inner_layers): ModuleList(
    (0-2): 3 x CollapsedCPLayer(
      (params_in): ReparamExp()
    )
    (3): SumLayer(
      (params): ReparamExp()
    )
  )
)


In [91]:
for param in pc.parameters(): 
    print (param.shape)

torch.Size([8, 1])
torch.Size([24, 2, 1, 1])
torch.Size([12, 2, 1, 1])
torch.Size([6, 2, 1, 1])
torch.Size([1, 6, 1])


In [92]:
data = Dataset("kin40k")
x_train, y_train, x_test, y_test = data.get_split(split=2)

kin40k dataset, N=40000, d=8


In [64]:
x_train.shape, x_test.shape

((36000, 8), (4000, 8))

In [93]:
x_train_real = x_train[:32000] #32000 # 2053   36584    36584     39063   13281    2672   # RE-RUN # 13279   # 1279   4701  824
y_train_real = y_train[:32000]
y_train_real = y_train_real.squeeze()
x_val = x_train[32000:]
y_val = y_train[32000:]
y_val = y_val.squeeze()
y_test = y_test.squeeze()

In [94]:
mean = x_train_real.mean(axis=0)
std = x_train_real.std(axis=0)

x_train_real_normalized = (x_train_real - mean) / std
x_val_normalized = (x_val - mean) / std
x_test_normalized = (x_test - mean) / std

In [95]:
import torch.nn as nn

class IdentityMapping(nn.Module):
    def __init__(self):
        super(IdentityMapping, self).__init__()
    
    def forward(self, x):
        return x

In [20]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Define the linear layer with input dimension 8 and output dimension 256
        self.linear = nn.Linear(in_features=8, out_features=128)
        # Define the ReLU activation function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Apply linear layer and then ReLU activation to the input x
        x = self.linear(x)
        x = self.relu(x)
        return x

In [96]:
np.random.seed(24)
torch.manual_seed(24) ####################### CHANGE

batch_size = 32

# X_train, y_train = make_data(n_samples)
# X_test, y_test = X_train, y_train

# x_train, y_train, x_test, y_test

ds_train = torch.utils.data.TensorDataset(torch.from_numpy(x_train_real).float(), torch.from_numpy(y_train_real).float())
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True) # suffle 

ds_val = torch.utils.data.TensorDataset(torch.from_numpy(x_val).float(), torch.from_numpy(y_val).float())
dl_val = torch.utils.data.DataLoader(ds_val, batch_size=512, shuffle=False)

ds_test = torch.utils.data.TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).float())
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=512, shuffle=False)

# steps = 5e3
epochs = 50
print(f"Training with {len(x_train_real)} datapoints for {epochs} epochs")

# Change this boolean to False for SNGP

input_dim = 8 # input di  # 128

num_outputs = 1 # regression with 1D output

feature_extractor = IdentityMapping()

n_inducing_points = 10
kernel = "HBF" ################# change 

initial_inducing_points, initial_lengthscale = initial_values(
        ds_train, feature_extractor, n_inducing_points
)

gp_model = CircuitGP(
    num_outputs=num_outputs,
    num_features=input_dim,          # CHANGE features / input_dim
    initial_lengthscale=initial_lengthscale,
    initial_inducing_points=initial_inducing_points,
    circuit=pc
    # kernel=kernel,
)
    
likelihood = GaussianLikelihood()
elbo_fn = VariationalELBO(likelihood, gp_model, num_data=len(ds_train))
loss_fn = lambda x, y: -elbo_fn(x, y)
    
# learning rate   
lr = 1e-5

parameters = [
    {"params": gp_model.parameters(), "lr": lr},
]

parameters.append({"params": likelihood.parameters(), "lr": lr})
    
    
optimizer = torch.optim.Adam(parameters)
pbar = ProgressBar()

step_counter = 0

def step(engine, batch):
    
    global step_counter
    step_counter += 1
    
    gp_model.train()
    likelihood.train()
    
    optimizer.zero_grad()
    
    x, y = batch
    if torch.cuda.is_available():
        x = x.cuda()
        y = y.cuda()

    y_pred = gp_model(x) # get y
    
    
    loss = loss_fn(y_pred, y) # loss
    # print("loss", loss)
    
    if torch.isnan(loss).any():
        print(f"Step {step_counter}: NaN detected in loss.")
        print("loss", loss)
        print("y_pred", y_pred)
    
    if torch.isnan(loss).any():
        print("NaN detected in loss, saving model and stopping.")
        # Save model weights before termination
        torch.save(gp_model.state_dict(), 'model_weights_before_nan.pt')
        engine.terminate()
        return
    
    loss.backward()
    optimizer.step()
    
    return loss.item()


def eval_step(engine, batch):
    gp_model.eval() # set to eval
    likelihood.eval()
    
    x, y = batch
    if torch.cuda.is_available():
        x = x.cuda()
        y = y.cuda()

    y_pred = gp_model(x)   
    return y_pred, y

    
trainer = Engine(step)
evaluator = Engine(eval_step)

metric = Average()
metric.attach(trainer, "loss")
pbar.attach(trainer)

metric = Loss(lambda y_pred, y: - likelihood.expected_log_prob(y, y_pred).mean())

metric.attach(evaluator, "loss")

@trainer.on(Events.EPOCH_COMPLETED(every=int(epochs/20) + 1))
def log_results(trainer):
    evaluator.run(dl_val) # val dataset
    print(f"Results - Epoch: {trainer.state.epoch} - "
          f"Val Loss: {evaluator.state.metrics['loss']:.2f} - "
          f"Train Loss: {trainer.state.metrics['loss']:.2f}")



Training with 32000 datapoints for 50 epochs
f_X_samples torch.Size([32000, 8])


/Users/irwinchay/opt/anaconda3/envs/cirkit1/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


initial_lengthscale tensor(3.8958)
All circuit parameters shape: 
torch.Size([8, 1])
torch.Size([24, 2, 1, 1])
torch.Size([12, 2, 1, 1])
torch.Size([6, 2, 1, 1])
torch.Size([1, 6, 1])
initial_lengthscale torch.Size([])
lengthscales shape torch.Size([8, 1])


In [97]:
for index, param in enumerate(gp_model.parameters()): 
    # if (index==2):
    print(param.shape)

torch.Size([10, 8])
torch.Size([10])
torch.Size([10, 10])
torch.Size([8, 1])
torch.Size([24, 2, 1, 1])
torch.Size([12, 2, 1, 1])
torch.Size([6, 2, 1, 1])
torch.Size([1, 6, 1])
torch.Size([1])
torch.Size([])


In [98]:
trainer.run(dl_train, max_epochs=epochs)

[1/1000]   0%|           [00:00<?]

[1/1000]   0%|           [00:00<?]

[1/1000]   0%|           [00:00<?]

Results - Epoch: 3 - Val Loss: 2757209472424265383936.00 - Train Loss: 2757453145030128566272.00


[1/1000]   0%|           [00:00<?]

...

Results - Epoch: 9 - Val Loss: 2751808899866310475776.00 - Train Loss: 2751924864741965561856.00


[1/1000]   0%|           [00:00<?]

Engine run is terminating due to exception: 


KeyboardInterrupt: 

: 

In [ ]:
for param in likelihood.parameters(): 
    print (param.shape)

In [59]:
from ignite.metrics import RootMeanSquaredError
import torch

# Assuming you have a function to compute RMSE, or you're using Ignite's RMSE metric

def eval_step(engine, batch):
    gp_model.eval()  # Ensure model is in evaluation mode
    likelihood.eval()
    
    x, y = batch
    if torch.cuda.is_available():
        x = x.cuda()
        y = y.cuda()

    # Assuming your model outputs a distribution, e.g., MultivariateNormal
    with torch.no_grad():  # Disable gradient computation for evaluation
        distribution = gp_model(x)
        y_pred = distribution.mean  # Use the mean of the distribution as the prediction

    return y_pred, y

# Update the evaluator engine
evaluator = Engine(eval_step)

# Attach the RMSE metric to the evaluator
rmse = RootMeanSquaredError()
rmse.attach(evaluator, "RMSE")

# After training, run the evaluator on the test dataset to compute the RMSE
evaluator.run(dl_test)

# Retrieve and display the RMSE
test_rmse = evaluator.state.metrics['RMSE']
print(f"Test RMSE: {test_rmse:.2f}")


Test RMSE: 0.86


In [ ]:
pc.input_layer.params.param.shape
# (self.num_vars, self.num_output_units, self.num_replicas, self.num_suff_stats)

In [ ]:
pc.scope_layer.scope.shape

In [ ]:
pc.inner_layers[0].params_in() #.param #.shape #.param.shape
# (F, H, I, O)
# (fold count, arity, input, output)

In [ ]:
from cirkit.models.rbf_kernel import RBFCircuitKernel

circuit_kernel = RBFCircuitKernel(pc, batch_shape=torch.Size([]))


In [ ]:
circuit_kernel(x1.squeeze(), x2.squeeze()).evaluate()

In [ ]:
x1.squeeze().shape

In [ ]:
# set parameters

pc.input_layer.params.param = torch.nn.Parameter(torch.log(torch.ones(tuple(pc.input_layer.params.shape))*3.3))
# pc.inner_layers[0].params_in.param = torch.nn.Parameter(torch.log(0.25*torch.ones(tuple(pc.inner_layers[0].params_in.shape))))
# pc.inner_layers[0].params_in = torch.nn.Parameter(torch.ones(tuple(pc.inner_layers[0].params_in.shape))*3.3)
# pc.inner_layers[1].params_in = torch.nn.Parameter(torch.ones(tuple(pc.inner_layers[1].params_in.shape))*3.3)
# pc.inner_layers[2].params_in = torch.nn.Parameter(torch.ones(tuple(pc.inner_layers[2].params_in.shape))*3.3)
# pc.inner_layers[3].params_in = torch.nn.Parameter(torch.ones(tuple(pc.inner_layers[3].params_in.shape))*3.3)

In [ ]:
pc.inner_layers[0].params_in() #.shape

In [ ]:
x1 = torch.randn(3, 8, 1)
x2 = torch.randn(3, 8, 1)

In [ ]:
pc(x1, x2).squeeze()

In [ ]:
def eval_pc(x1, x2): 
    return pc(x1.unsqueeze(-1), x2.unsqueeze(-1)).squeeze(-1)

eval_pc(x1.squeeze(), x2.squeeze())

In [ ]:
from gpytorch.kernels import RBFKernel

# x = torch.randn(3, 5)
covar_module = RBFKernel()
covar_module.lengthscale = torch.tensor(3.3)
covar_module(x1.squeeze(), x2.squeeze()).evaluate()

In [ ]:
x1.squeeze().shape

In [ ]:
from gpytorch.kernels import RBFKernel
x = torch.randn(3, 2)
RBFKernel().lengthscale = torch.tensor(3.3)

In [ ]:
# Test RBF input output = RBF kernel 

In [ ]:
from gpytorch.kernels import RBFKernel, SpectralMixtureKernel

x = torch.randn(3, 5)
covar_module = SpectralMixtureKernel(num_mixtures=2, ard_num_dims=5)
covar_module.mixture_scales = torch.tensor(3.3).expand(1, 2, 1, 5)
covar_module.mixture_means = torch.tensor(2.2).expand(1, 2, 1, 5)
covar_module.mixture_weights = torch.tensor([0.5]).expand(1, 2, 1, 5)
covar_module(x).evaluate()
# covar_module.lengthscale

In [ ]:
from cirkit.layers.input.sm_kernel import SMKernelLayer
input_la = SMKernelLayer(num_vars=5, num_output_units=1)

input_la.params = torch.nn.Parameter(torch.ones((5,1))*3.3)

# input_la(x1, x2).squeeze().shape

# input_la(x.unsqueeze(-1), x.unsqueeze(-1)).shape

torch.prod(torch.exp(input_la(x.unsqueeze(-1), x.unsqueeze(-1)).squeeze()), dim=2)

In [ ]:
input_la = RBFKernelLayer(num_vars=20, num_output_units=1)

input_la.params = torch.nn.Parameter(torch.ones((20,1))*3.3)

# input_la(x1, x2).squeeze().shape
torch.prod(input_la(x1, x1).squeeze(), dim=2)

In [ ]:
from gpytorch.kernels import RBFKernel

x = torch.randn(3, 5)
covar_module = RBFKernel()
covar_module.lengthscale = torch.tensor(3.3)
covar_module(x).evaluate()
# covar_module.lengthscale

In [ ]:
from cirkit.layers.input.rbf_kernel import RBFKernelLayer
input_la = RBFKernelLayer(num_vars=5, num_output_units=1)

input_la.params.param = torch.nn.Parameter(torch.log(torch.ones(tuple(input_la.params.shape))*3.3))
# pc.input_layer.params.param = torch.nn.Parameter(torch.log(torch.ones(tuple(pc.input_layer.params.shape))*3.3))

# input_la(x1, x2).squeeze().shape

# input_la(x.unsqueeze(-1), x.unsqueeze(-1)).shape

torch.prod(torch.exp(input_la(x.unsqueeze(-1), x.unsqueeze(-1)).squeeze()), dim=2)

In [ ]:
train_x = torch.linspace(0, 1, 3)
torch.sin(train_x * (2 * math.pi))

In [ ]:
import math


# train_x = torch.linspace(0, 1, 3)
# train_y = torch.sin(train_x * (2 * math.pi))
train_x = torch.rand((3, 5))
train_y = torch.rand((3))

covar_module = SpectralMixtureKernel(num_mixtures=4, ard_num_dims=5)
covar_module.initialize_from_data(train_x, train_y)
covar_module(train_x).evaluate()

In [ ]:
from cirkit.layers.input.sm_kernel import SMKernelLayer
input_la = SMKernelLayer(num_vars=1, num_output_units=4)

input_la.params_mu.param = torch.nn.Parameter(covar_module.mixture_means)
input_la.params_sigma.param = torch.nn.Parameter(torch.log(covar_module.mixture_scales))


to_be_weighted = input_la(train_x.unsqueeze(-1), train_x.unsqueeze(-1))

to_be_weighted = torch.prod(to_be_weighted, dim=2, keepdim=True) / 5

tensor1_expanded = covar_module.mixture_weights.expand_as(to_be_weighted.squeeze(-1))

# Element-wise multiplication and then sum over the inner product dimension (dimension 3 after squeeze)
(tensor1_expanded * to_be_weighted.squeeze(-1)).sum(dim=3).squeeze()

# torch.prod(finalfinal, dim=-1, keepdim=False)

In [ ]:
to_be_weighted.shape

In [ ]:
covar_module.mixture_scales.shape

In [ ]:
covar_module(x1).evaluate().shape

In [ ]:
train_x.unsqueeze(-1).unsqueeze(-1).shape

In [ ]:
x_2 = torch.tensor([[-0.6281], [ 0.1011], [ 0.0664]])

In [ ]:
from cirkit.layers.input.rbf_kernel import RBFKernelLayer
input_la = RBFKernelLayer(num_vars=2, num_output_units=1)

input_la.params = torch.nn.Parameter(torch.ones((1,1))*3.3)

input_la(x_2.unsqueeze(-1), x_2.unsqueeze(-1)).squeeze()

In [ ]:
input_la.params

In [ ]:
torch.ones((2,1))*3.3

In [ ]:
train_x.unsqueeze(-1).unsqueeze(-1).shape

In [ ]:
input_la(x.unsqueeze(-1), x.unsqueeze(-1)).squeeze()

In [ ]:
x_2.unsqueeze(-1).shape

In [ ]:
torch.cdist(x1, x2, p=2)

In [ ]:
from torch import optim
from torch.utils.data import DataLoader
train_dataloader = DataLoader(data_train, shuffle=True, batch_size=256)
test_dataloader = DataLoader(data_test, shuffle=False, batch_size=256)
optimizer = optim.SGD(pc.parameters(), lr=0.1, momentum=0.9)

Since the constructed PC is not necessarily normalized, we construct the integral circuit that will compute the partition function. Note that parameters are shared and therefore there is no additional memory required.

In [ ]:
from cirkit.models.functional import integrate
pc_pf = integrate(pc)

Finally, we optimize the parameters for 5 epochs by minimizing the negative log-likelohood.

In [ ]:
num_epochs = 5
for epoch_idx in range(num_epochs):
    running_loss = 0.0
    for batch, _ in train_dataloader:
        batch = batch.to(device).unsqueeze(dim=-1)  # Add a channel dimension
        log_score = pc(batch)
        log_pf = pc_pf(batch)     # Compute the partition function
        lls = log_score - log_pf  # Compute the log-likelihood
        loss = -torch.mean(lls)   # The loss is the negative average log-likelihood
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss * len(batch)
        # Clamp the parameters to ensure they are in the intended domain
        # This is needed if we do not use any reparametrization to ensure parameters non-negativity
        # In our case, clamping is disable becuase we reparameterize via exponentiation (see above)
        #for layer in model.inner_layers:
        #    layer.clamp_params()
    print(f"Epoch {epoch_idx}: Average NLL: {running_loss / len(data_train):.3f}")

We then evaluate our model on test data by computing the average log-likelihood and bits per dimension.

In [ ]:
with torch.no_grad():
    pc.eval()
    log_pf = pc_pf(torch.empty((), device=device))  # Compute the partition function once for testing
    test_lls = 0.0
    for batch, _ in test_dataloader:
        log_score = pc(batch.to(device).unsqueeze(dim=-1))
        lls = log_score - log_pf
        test_lls += lls.sum().item()
    average_ll = test_lls / len(data_test)
    bpd = -average_ll / (num_variables * np.log(2.0))
    print(f"Average test LL: {average_ll:.3f}")
    print(f"Bits per dimension: {bpd}")

In [ ]:
#!/usr/bin/env python3

import gpytorch

# from ..functions import RBFCovariance
# from ..settings import trace_mode
from gpytorch.kernels import Kernel


def postprocess_rbf(dist_mat):
    return dist_mat.div_(-2).exp_()


class TestRBFKernel(Kernel):
    r"""
    Computes a covariance matrix based on the RBF (squared exponential) kernel
    between inputs :math:`\mathbf{x_1}` and :math:`\mathbf{x_2}`:

    .. math::

       \begin{equation*}
          k_{\text{RBF}}(\mathbf{x_1}, \mathbf{x_2}) = \exp \left( -\frac{1}{2}
          (\mathbf{x_1} - \mathbf{x_2})^\top \Theta^{-2} (\mathbf{x_1} - \mathbf{x_2}) \right)
       \end{equation*}

    where :math:`\Theta` is a :attr:`lengthscale` parameter.
    See :class:`gpytorch.kernels.Kernel` for descriptions of the lengthscale options.

    .. note::

        This kernel does not have an `outputscale` parameter. To add a scaling parameter,
        decorate this kernel with a :class:`gpytorch.kernels.ScaleKernel`.

    Args:
        :attr:`ard_num_dims` (int, optional):
            Set this if you want a separate lengthscale for each
            input dimension. It should be `d` if :attr:`x1` is a `n x d` matrix. Default: `None`
        :attr:`batch_shape` (torch.Size, optional):
            Set this if you want a separate lengthscale for each
            batch of input data. It should be `b` if :attr:`x1` is a `b x n x d` tensor. Default: `torch.Size([])`.
        :attr:`active_dims` (tuple of ints, optional):
            Set this if you want to compute the covariance of only a few input dimensions. The ints
            corresponds to the indices of the dimensions. Default: `None`.
        :attr:`lengthscale_prior` (Prior, optional):
            Set this if you want to apply a prior to the lengthscale parameter.  Default: `None`.
        :attr:`lengthscale_constraint` (Constraint, optional):
            Set this if you want to apply a constraint to the lengthscale parameter. Default: `Positive`.
        :attr:`eps` (float):
            The minimum value that the lengthscale can take (prevents divide by zero errors). Default: `1e-6`.

    Attributes:
        :attr:`lengthscale` (Tensor):
            The lengthscale parameter. Size/shape of parameter depends on the
            :attr:`ard_num_dims` and :attr:`batch_shape` arguments.

    Example:
        >>> x = torch.randn(10, 5)
        >>> # Non-batch: Simple option
        >>> covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        >>> # Non-batch: ARD (different lengthscale for each input dimension)
        >>> covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=5))
        >>> covar = covar_module(x)  # Output: LazyTensor of size (10 x 10)
        >>>
        >>> batch_x = torch.randn(2, 10, 5)
        >>> # Batch: Simple option
        >>> covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        >>> # Batch: different lengthscale for each batch
        >>> covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(batch_shape=torch.Size([2])))
        >>> covar = covar_module(x)  # Output: LazyTensor of size (2 x 10 x 10)
    """

    has_lengthscale = True

    def forward(self, x1, x2, diag=False, **params):

        x1_ = x1.div(self.lengthscale)
        x2_ = x2.div(self.lengthscale)
        
        # print ("x1, x2", x1_, x2_)
        
        return self.covar_dist(
            x1_, x2_, square_dist=True, diag=diag, dist_postprocess_func=postprocess_rbf, postprocess=True, **params
        )

In [ ]:
test_kernel = TestRBFKernel()
test_kernel.lengthscale = torch.tensor(3.3)

In [ ]:
test_kernel.lengthscale

In [ ]:
test_kernel(x1.squeeze(),x2.squeeze()).evaluate()

In [ ]:
x1.shape